In [8]:
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
import numpy as np

import os
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [4]:
data_folder = r'getting_the_data\data'

In [44]:
#creating folder to save the backttests
backtest_folder = r'Backtest'
if not os.path.exists(backtest_folder):
    os.makedirs(backtest_folder)

In [45]:
#making parameters grid
parameters_grid = []
for entry_momentum in reversed(range(2,6)):
    for exit_momentum in reversed(range(1,entry_momentum)): #this is also entry_pnl
        for exit_pnl in reversed(range(0,exit_momentum)):
            parameters_grid.append((entry_momentum,exit_momentum,exit_pnl))

In [54]:
#trade params
df_symbols = pd.read_csv(r'getting_the_data\df_symbols.csv', encoding='utf-8')

In [48]:
for filename in [x for x in os.listdir(data_folder) if '.csv' in x]:
    ticker = filename.replace('.csv','')
    
     #trade exchange information
    single_point_value = df_symbols[df_symbols['Symbol']==ticker]['Single point value']\
                     .values[0]
    ticks = df_symbols[df_symbols['Symbol']==ticker]['Ticks']\
                     .values[0]
    #importing price history
    df = pd.read_csv(data_folder+r'\\'+filename, encoding='utf-8').drop('Unnamed: 0',1)
    df['timestamp'] = df[['date','time']].apply(lambda x:\
                          pd.to_datetime(x[0]+x[1].split('days')[1]), axis=1) 
    df['date'] = df['timestamp'].dt.date
    df['time'] = df['time'].apply(lambda x: pd.to_datetime(x.split('days')[1].strip()).time())
        
    df = df.sort_values(by = 'timestamp')
    df = df.reset_index(drop=True)
    
    #MAKING A DF, DATE AS AN INDEX PRICES AS COLUMNS
    #QUESTIONS: IT SEEMS THAT THE BAR CLOSE IS NOT EQUAL TO NEXT BAR OPEN, IS THIS APPROACH CORRECT?
    df_features = pd.DataFrame()

    settlement_time = df['time'].max()
    for time in df['time'].unique():
        minute_change = str((dt.datetime.combine(dt.date.today(), settlement_time) \
                        - dt.datetime.combine(dt.date.today(), time)).seconds/60+1)\
                        .split('.')[0]
        if minute_change != '1':

            df_part = df[df['time']==time].set_index('date')[['open_p']].copy()
            df_part.columns = ['price_'+minute_change]
        else:
            df_part = df[df['time']==time].set_index('date')[['open_p','close_p']].copy()
            df_part.columns = ['price_'+minute_change,'price_0']

        df_features = df_features.join(df_part, how='outer')
     
    #RUNNING BACKTEST AND STORING RESULTS
    results = pd.DataFrame()
    for parameters in parameters_grid:
        entry_momentum = 'price_'+str(parameters[0])
        exit_momentum = 'price_'+str(parameters[1])
        exit_pnl = 'price_'+str(parameters[2])

        #calculating momentum
        df_features['momentum'] = (df_features[exit_momentum]/df_features[entry_momentum]-1)*100

        for momentum_threshold in \
        df_features['momentum'].quantile([0.1,0.2,0.3,0.6,0.7,0.8,0.9]).values:
            if momentum_threshold>0:
                #go long
                df_features['pnl'] = df_features[['momentum',exit_momentum,exit_pnl]].\
                                     apply(lambda x: x[2]-x[1] if x[0]>momentum_threshold else np.nan, axis=1)\
                                     *single_point_value
            else:
                #go short
                df_features['pnl'] = df_features[['momentum',exit_momentum,exit_pnl]].\
                                     apply(lambda x: x[1]-x[2] if x[0]<momentum_threshold else np.nan, axis=1)\
                                     *single_point_value

            #calculating stats
            summary = {}
            summary['entry_momentum'] = str(parameters[0])
            summary['exit_momentum'] = str(parameters[1])
            summary['exit_pnl'] = str(parameters[2])
            summary['momentum_threshold'] = momentum_threshold
            summary['total_days'] = df_features.shape[0]
            summary['days_traded'] = df_features[pd.notnull(df_features['pnl'])].shape[0]
            summary['total_pnl'] = df_features['pnl'].sum()
            summary['mean_pnl'] = df_features['pnl'].mean()
            summary['sharpe_ratio'] = (df_features['pnl'].mean()/df_features['pnl'].std())*(252**0.5)
            results = pd.concat([results,pd.DataFrame(summary, index=[0])])
    results['ticker'] = ticker
    results.sort_values(by='sharpe_ratio').to_csv(\
    backtest_folder+r'\\'+ticker+'.csv', encoding='utf-8', index=False)

# Analyzing the most successful strategies

In [49]:
df = pd.DataFrame()
for file_name in [x for x in os.listdir(backtest_folder) if '.csv' in x]:
    df_part = pd.read_csv(backtest_folder+r'\\'+file_name, encoding='utf-8')
    df = pd.concat([df,df_part])

In [51]:
df.sort_values(by='sharpe_ratio').tail(10)

,entry_momentum,exit_momentum,exit_pnl,momentum_threshold,total_days,days_traded,total_pnl,mean_pnl,sharpe_ratio,ticker
139,4,2,0,0.052,811,81,54.300,0.670,4.788,@MFS#
139,5,4,3,0.114,806,46,50.650,1.101,4.803,QPA#
139,5,3,1,-0.016,815,77,0.550,0.007,5.040,@DX#
139,4,2,1,0.164,842,84,19.250,0.229,5.159,GAS#
138,5,4,1,0.101,822,82,7.600,0.093,5.168,@KC#
139,5,4,1,0.012,815,82,0.009,0.000,5.240,@EU#
139,4,3,2,0.138,822,83,93.000,1.120,5.602,@CC#
139,5,4,2,0.101,822,82,6.000,0.073,5.861,@KC#
138,5,3,0,0.015,815,82,0.000,0.000,5.965,@JY#
139,5,3,1,0.015,815,82,0.000,0.000,6.315,@JY#


In [52]:
df.sort_values(by='mean_pnl').tail(10)

,entry_momentum,exit_momentum,exit_pnl,momentum_threshold,total_days,days_traded,total_pnl,mean_pnl,sharpe_ratio,ticker
126,5,2,0,-0.058,812,163,"1,321.000",8.104,2.670,@YM#
118,4,2,0,-0.071,812,82,687.000,8.378,2.106,@YM#
129,5,4,0,-0.024,812,244,"2,116.000",8.672,2.719,@YM#
133,3,1,0,0.064,812,82,731.000,8.915,3.066,@YM#
123,5,2,0,-0.097,812,82,750.000,9.146,2.404,@YM#
127,5,2,0,0.095,812,82,791.000,9.646,2.699,@YM#
134,5,4,1,-0.076,812,82,802.000,9.780,3.070,@YM#
135,5,4,0,-0.042,812,163,"1,957.000",12.006,3.251,@YM#
125,5,4,0,-0.076,812,82,"1,040.000",12.683,2.641,@YM#
139,5,1,0,0.101,812,82,"1,058.000",12.902,4.753,@YM#


In [53]:
df.sort_values(by='total_pnl').tail(10)

,entry_momentum,exit_momentum,exit_pnl,momentum_threshold,total_days,days_traded,total_pnl,mean_pnl,sharpe_ratio,ticker
139,5,1,0,0.101,812,82,"1,058.000",12.902,4.753,@YM#
119,4,2,0,-0.040,812,163,"1,123.000",6.890,2.234,@YM#
130,5,4,1,-0.042,812,163,"1,134.000",6.957,2.761,@YM#
137,5,1,0,0.061,812,163,"1,185.000",7.270,3.283,@YM#
114,5,2,0,0.027,812,244,"1,230.000",5.041,2.005,@YM#
106,5,2,0,0.011,812,325,"1,271.000",3.911,1.742,@YM#
126,5,2,0,-0.058,812,163,"1,321.000",8.104,2.670,@YM#
122,5,2,0,-0.036,812,244,"1,512.000",6.197,2.342,@YM#
135,5,4,0,-0.042,812,163,"1,957.000",12.006,3.251,@YM#
129,5,4,0,-0.024,812,244,"2,116.000",8.672,2.719,@YM#
